In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

Defining the Adjacency Matrix (simetrically normalized)

In [2]:
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device name:", torch.cuda.get_device_name(0))

CUDA available: True
Device name: NVIDIA A100-SXM4-40GB


In [3]:

def normalize_adjacency_dense_gpu(A):
    """
    Normalize adjacency matrix on GPU.
    A: Dense adjacency matrix (torch.Tensor).
    """
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    A = A.to(device)  # Move to GPU if available
    
    # Ensure self-loops
    #A = A + torch.eye(A.size(0), device=A.device)
    
    # Degree vector
    row_sum = torch.sum(A, dim=1)
    print(row_sum[0])
    
    # Avoid division by zero by adding a small epsilon
    D_inv_sqrt = torch.diag(1.0 / torch.sqrt(1e-10+ row_sum ))
    print(D_inv_sqrt[0])
    # Normalize adjacency
    normalized_A = D_inv_sqrt @ A @ D_inv_sqrt
    
    # Enforce symmetry (optional but helps to handle numerical instability)
    normalized_A = (normalized_A + normalized_A.T) / 2.0
    
    return normalized_A


Define the GCN Layers

In [4]:
class GCNLayer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(GCNLayer, self).__init__()
        self.weight = nn.Parameter(torch.randn(input_dim, output_dim))
        #self.weight = nn.Parameter(torch.randn(output_dim, input_dim))  # output_dim should be first


    def forward(self, X, A_tilde):
        return A_tilde @ X @ self.weight


Inference Model

In [5]:
class InferenceModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(InferenceModel, self).__init__()
        self.gcn1 = GCNLayer(input_dim, hidden_dim)
        self.gcn2_mu = GCNLayer(hidden_dim, latent_dim)
        self.gcn2_logsigma = GCNLayer(hidden_dim, latent_dim)

    def forward(self, X, A_tilde):
        H = F.relu(self.gcn1(X, A_tilde))  # Shared first layer
        mu = self.gcn2_mu(H, A_tilde)  # Mean matrix
        log_sigma = self.gcn2_logsigma(H, A_tilde)  # Log-variance matrix
        return mu, log_sigma


Variational Grpah Auto-Encoder

In [6]:
class VGAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(VGAE, self).__init__()
        self.encoder = InferenceModel(input_dim, hidden_dim, latent_dim)

    def forward(self, X, A_tilde):
        mu, log_sigma = self.encoder(X, A_tilde)
        # Reparameterization trick
        std = torch.exp(0.5 * log_sigma)
        std = torch.clamp(std, max=1e5) 
        eps = torch.randn_like(std)
        Z = mu + eps * std
        Z = F.normalize(Z, p=2, dim=1)  # Normalize rows of Z to unit length

        A_reconstructed = torch.sigmoid(torch.matmul(Z, Z.T))
        return Z, A_reconstructed, mu, log_sigma


In [7]:

class VGAE_MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super().__init__()
        self.encoder = InferenceModel(input_dim, hidden_dim, latent_dim)
        
        # MLP Decoder (2-layer perceptron)
        self.decoder = nn.Sequential(
            nn.Linear(2 * latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, hidden_dim // 4),
            nn.ReLU(),
            nn.Linear(hidden_dim // 4, 1)
            )
        

    def forward(self, X, A_tilde, edge_index=None):
        mu, log_sigma = self.encoder(X, A_tilde)
        
        # Reparameterization trick
        std = torch.exp(0.5 * log_sigma)
        std = torch.clamp(std, max=1e5) 
        eps = torch.randn_like(std)
        Z = mu + eps * std
        Z = F.normalize(Z, p=2, dim=1)  # Normalize rows of Z to unit length
        
        batch_size = 10000  # Adjust based on memory
        num_edges = edge_index.shape[1]
        A_reconstructed_list = []

        for i in range(0, num_edges, batch_size):
            batch_edges = edge_index[:, i : i + batch_size]
            src, dst = batch_edges
            Z_concat = torch.cat([Z[src], Z[dst]], dim=1)
            A_reconstructed_list.append(torch.sigmoid(self.decoder(Z_concat)).squeeze())

        A_reconstructed = torch.cat(A_reconstructed_list)
        print(A_reconstructed.shape)
        return Z, A_reconstructed, mu, log_sigma


In [8]:
class WeightedInnerProductDecoder(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(latent_dim))  # Learnable weight vector

    def forward(self, Z):
        Z_weighted = Z * self.weight  # Apply element-wise weight
        A_reconstructed = torch.sigmoid(torch.matmul(Z, Z_weighted.T))  # Full adjacency matrix
        return A_reconstructed


class VGAE_W(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super().__init__()
        self.encoder = InferenceModel(input_dim, hidden_dim, latent_dim)
        self.decoder = WeightedInnerProductDecoder(latent_dim)

    def forward(self, X, A_tilde):
        mu, log_sigma = self.encoder(X, A_tilde)
        
        # Reparameterization trick
        std = torch.exp(0.5 * log_sigma)
        std = torch.clamp(std, max=1e5)
        eps = torch.randn_like(std)
        Z = mu + eps * std
        Z = F.normalize(Z, p=2, dim=1)  # Normalize rows of Z to unit length

        A_reconstructed = self.decoder(Z)  # No need for edge_index now

        return Z, A_reconstructed, mu, log_sigma


Loss Function

In [9]:
def loss_function(A, A_reconstructed, mu, log_sigma):
    # Reconstruction loss (Binary Cross-Entropy)
    recon_loss = F.binary_cross_entropy(A_reconstructed, A, reduction='sum')

    # KL Divergence
    kl_loss = -0.5 * torch.sum(1 + log_sigma - mu.pow(2) - log_sigma.clamp(max=10).exp())

    return recon_loss + kl_loss


In [10]:
def loss_function_mlp(A, A_reconstructed, mu, log_sigma, edge_index):
    src, dst = edge_index  # Edge indices

    # If A_reconstructed is 1D, select indices correctly
    A_pred = A_reconstructed[torch.arange(edge_index.shape[1])]

    # Get true adjacency values
    A_true = A[src, dst]

    # BCE loss
    recon_loss = F.binary_cross_entropy(A_pred, A_true, reduction='sum')

    # KL Divergence
    kl_loss = -0.5 * torch.sum(1 + log_sigma - mu.pow(2) - log_sigma.clamp(max=10).exp())

    return recon_loss + kl_loss

# Training

## VGAE

### Cora

In [11]:
from torch_geometric.datasets import CoraFull

cora_dataset = CoraFull(root='GraphDatasets/Cora')
print(f'Dataset: {cora_dataset}:')
print('======================')
print(f'Number of graphs: {len(cora_dataset)}')
print(f'Number of features: {cora_dataset.num_features}')
print(f'Number of classes: {cora_dataset.num_classes}')

/home/scur2863/graphs/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset: CoraFull():
Number of graphs: 1
Number of features: 8710
Number of classes: 70


In [12]:
import torch

# Graph data
data = cora_dataset[0]  
X = data.x  # features matrix (N x D)
edge_index = data.edge_index  # Edge list (2 x E)

# Create the adjacency matrix (A)
num_nodes = X.size(0)
A = torch.zeros((num_nodes, num_nodes))

# Convert the edge_index to an adjacency matrix
row, col = edge_index
A[row, col] = 1
A[col, row] = 1  # Since the graph is undirected

# Optionally, add self-loops (diagonal elements set to 1)
A.fill_diagonal_(1)

print("Adjacency matrix (A):", A)
print("Node feature matrix (X):", X)


Adjacency matrix (A): tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])
Node feature matrix (X): tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [13]:
import random
import networkx as nx
from itertools import combinations
import torch
from torch_geometric.utils import to_networkx, from_networkx
from sklearn.model_selection import train_test_split

def remove_edges_and_sample_optimized(edge_index, num_nodes, test_size=0.1, val_size=0.05):
    """
    Optimized version of edge removal and sampling non-edges.
    
    Parameters:
    - edge_index: Tensor (2, E) containing edges in COO format.
    - num_nodes: int, number of nodes in the graph.
    - test_size: float, fraction of edges to use for the test set.
    - val_size: float, fraction of edges to use for the validation set.

    Returns:
    - train_edge_index: Tensor containing edges for training.
    - val_edges: List of validation edges.
    - test_edges: List of test edges.
    - val_non_edges: List of validation non-edges.
    - test_non_edges: List of test non-edges.
    """
    # Convert edge_index to a set of edges for faster lookup
    edges = set(map(tuple, edge_index.t().tolist()))
    
    # Generate all possible node pairs (i, j) for non-edges
    all_pairs = set(combinations(range(num_nodes), 2))
    non_edges = list(all_pairs - edges)

    # Split edges into validation and test sets
    edges = list(edges)
    train_edges, temp_edges = train_test_split(edges, test_size=test_size + val_size, random_state=42)
    val_edges, test_edges = train_test_split(temp_edges, test_size=test_size / (test_size + val_size), random_state=42)

    # Sample non-edges for validation and test sets
    num_val_non_edges = len(val_edges)
    num_test_non_edges = len(test_edges)

    val_non_edges = random.sample(non_edges, num_val_non_edges)
    test_non_edges = random.sample(non_edges, num_test_non_edges)
    # Recreate the training graph without validation and test edges
    train_graph = nx.Graph()
    train_graph.add_edges_from(train_edges)
    train_graph.add_nodes_from(range(num_nodes))  # Add isolated nodes
    train_edge_index = torch.tensor(list(train_graph.edges)).t().contiguous()

    # Recreate training edge_index
    train_edge_index = torch.tensor(train_edges).t().contiguous()

    return train_edge_index, val_edges, test_edges, val_non_edges, test_non_edges,train_graph

In [14]:
# Extract edge_index and number of nodes
edge_index = data.edge_index  # (2, E)
num_nodes = data.num_nodes

# Split edges and sample non-edges
train_edge_index, val_edges, test_edges, val_non_edges, test_non_edges, train_graph = remove_edges_and_sample_optimized(edge_index, num_nodes)

print("Train edge index shape:", train_edge_index.shape)
print("Number of validation edges:", len(val_edges))
print("Number of test edges:", len(test_edges))
print("Number of validation non-edges:", len(val_non_edges))
print("Number of test non-edges:", len(test_non_edges))


Train edge index shape: torch.Size([2, 107815])
Number of validation edges: 6342
Number of test edges: 12685
Number of validation non-edges: 6342
Number of test non-edges: 12685


#### Normal

In [19]:
import torch
from torch_geometric.utils import to_dense_adj

# Extract train graph adjacency matrix
num_n=train_edge_index.max().item() + 1
train_adj_matrix = to_dense_adj(train_edge_index, max_num_nodes=num_n)[0]  # Convert to dense adjacency matrix
train_adj_matrix = train_adj_matrix.to(torch.float32)  # Ensure float type for computations
train_adj_matrix = to_dense_adj(train_edge_index, max_num_nodes=num_nodes)[0] 
# Convert to SciPy sparse matrix
# Create the adjacency matrix from the edge list (train_edge_index)
#train_adj_matrix = to_dense_adj(train_edge_index, max_num_nodes=num_nodes)[0]

# Enforce symmetry (add the transpose to ensure both directions are captured)
#train_adj_matrix = train_adj_matrix + train_adj_matrix.T

# Ensure that the diagonal entries are 1 (self-loops)
#train_adj_matrix.fill_diagonal_(1.0)
#train_adj_matrix = train_adj_matrix.clamp(max=1)
# Node features
if data.x is not None:
    X = data.x  # Use provided node features
else:
    X = torch.eye(num_nodes)  # Use identity matrix if featureless

# Normalize adjacency for training graph
A_tilde_train = normalize_adjacency_dense_gpu(train_adj_matrix)

# Initialize model
input_dim = X.shape[1]
hidden_dim = 32
latent_dim = 16
model = VGAE(input_dim, hidden_dim, latent_dim)

model = model.to('cuda')  # Move model to GPU if available
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

tensor(19., device='cuda:0')
tensor([0.2294, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000], device='cuda:0')


#### Weighted

In [18]:
import torch
from torch_geometric.utils import to_dense_adj

# Extract train graph adjacency matrix
num_n=train_edge_index.max().item() + 1
train_adj_matrix = to_dense_adj(train_edge_index, max_num_nodes=num_n)[0]  # Convert to dense adjacency matrix
train_adj_matrix = train_adj_matrix.to(torch.float32)  # Ensure float type for computations
train_adj_matrix = to_dense_adj(train_edge_index, max_num_nodes=num_nodes)[0] 
# Convert to SciPy sparse matrix
# Create the adjacency matrix from the edge list (train_edge_index)
#train_adj_matrix = to_dense_adj(train_edge_index, max_num_nodes=num_nodes)[0]

# Enforce symmetry (add the transpose to ensure both directions are captured)
train_adj_matrix = train_adj_matrix + train_adj_matrix.T

# Ensure that the diagonal entries are 1 (self-loops)
train_adj_matrix.fill_diagonal_(1.0)
train_adj_matrix = train_adj_matrix.clamp(max=1)
# Node features
if data.x is not None:
    X = data.x  # Use provided node features
else:
    X = torch.eye(num_nodes)  # Use identity matrix if featureless

# Normalize adjacency for training graph
A_tilde_train = normalize_adjacency_dense_gpu(train_adj_matrix)

# Initialize model
input_dim = X.shape[1]
hidden_dim = 32
latent_dim = 16
model = VGAE_W(input_dim, hidden_dim, latent_dim)

model = model.to('cuda')  # Move model to GPU if available
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [26]:
val_edge = torch.tensor(val_edges).contiguous()
val_adj_matrix = to_dense_adj(val_edge, max_num_nodes=val_edge.shape[0])[0]  # Convert to dense adjacency matrix
val_adj_matrix = val_adj_matrix.to(torch.float32)  # Ensure float type for computations

A_tilde_val = normalize_adjacency_dense_gpu(val_adj_matrix)

In [28]:
import matplotlib.pyplot as plt

num_epochs = 200
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass on training set
    Z, A_reconstructed, mu, log_sigma = model(X.to('cuda'), A_tilde_train.to('cuda'))
    
    # Clamp log_sigma
    log_sigma = torch.clamp(log_sigma, min=-18, max=18)

    # Compute training loss
    train_loss = loss_function(train_adj_matrix.to('cuda'), A_reconstructed.to('cuda'), mu, log_sigma)
    train_loss.backward()
    optimizer.step()

    # Store train loss
    train_losses.append(train_loss.item())

    # Validation step
    model.eval()
    with torch.no_grad():
        Z_val, A_reconstructed_val, mu_val, log_sigma_val = model(X.to('cuda'), A_tilde_val.to('cuda'))
        log_sigma_val = torch.clamp(log_sigma_val, min=-18, max=18)
        val_loss = loss_function(val_adj_matrix.to('cuda'), A_reconstructed_val.to('cuda'), mu_val, log_sigma_val)
        val_losses.append(val_loss.item())

    print(f"Epoch {epoch + 1}, Train Loss: {train_loss.item():.4f}, Val Loss: {val_loss.item():.4f}")

# Plot Training & Validation Loss
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs + 1), train_losses, label="Train Loss", color="blue")
plt.plot(range(1, num_epochs + 1), val_losses, label="Validation Loss", color="red", linestyle="--")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.grid()
plt.show()


RuntimeError: mat1 and mat2 shapes cannot be multiplied (6342x6342 and 19793x8710)

In [21]:
num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    Z, A_reconstructed, mu, log_sigma = model(X.to('cuda'), A_tilde_train.to('cuda'))
    
    # Clamp log_sigma to prevent extreme values
    log_sigma = torch.clamp(log_sigma, min=-18, max=18)
    
    # Compute loss
    loss = loss_function(train_adj_matrix.to('cuda'), A_reconstructed.to('cuda'), mu, log_sigma)
    loss.backward()

    # Update parameters
    optimizer.step()

    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")



Epoch 1, Loss: 1.3056686159876943e+25


../aten/src/ATen/native/cuda/Loss.cu:94: operator(): block: [44,0,0], thread: [0,0,0] Assertion `input_val >= zero && input_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:94: operator(): block: [44,0,0], thread: [1,0,0] Assertion `input_val >= zero && input_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:94: operator(): block: [44,0,0], thread: [2,0,0] Assertion `input_val >= zero && input_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:94: operator(): block: [44,0,0], thread: [3,0,0] Assertion `input_val >= zero && input_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:94: operator(): block: [44,0,0], thread: [4,0,0] Assertion `input_val >= zero && input_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:94: operator(): block: [44,0,0], thread: [5,0,0] Assertion `input_val >= zero && input_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:94: operator(): block: [44,0,0], thread: [6,0,0] Assertion `input_val >= zero && input_val <= one` 

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [16]:
import matplotlib.pyplot as plt

num_epochs = 150
train_losses = []

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    Z, A_reconstructed, mu, log_sigma = model(X.to('cuda'), A_tilde_train.to('cuda'))
    
    # Clamp log_sigma to prevent extreme values
    log_sigma = torch.clamp(log_sigma, min=-18, max=18)
    
    # Compute loss
    loss = loss_function(train_adj_matrix.to('cuda'), A_reconstructed.to('cuda'), mu, log_sigma)
    loss.backward()

    # Update parameters
    optimizer.step()

    # Store loss for plotting
    train_losses.append(loss.item())

    print(f"Epoch {epoch + 1}, Loss: {loss.item():.4f}")

# Plot Training Loss
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs + 1), train_losses, label="Train Loss", color="blue")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss Over Epochs")
plt.legend()
plt.grid()
plt.show()


Epoch 1, Loss: 1914470662144.0000


../aten/src/ATen/native/cuda/Loss.cu:94: operator(): block: [26,0,0], thread: [96,0,0] Assertion `input_val >= zero && input_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:94: operator(): block: [26,0,0], thread: [97,0,0] Assertion `input_val >= zero && input_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:94: operator(): block: [26,0,0], thread: [98,0,0] Assertion `input_val >= zero && input_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:94: operator(): block: [26,0,0], thread: [99,0,0] Assertion `input_val >= zero && input_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:94: operator(): block: [26,0,0], thread: [100,0,0] Assertion `input_val >= zero && input_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:94: operator(): block: [26,0,0], thread: [101,0,0] Assertion `input_val >= zero && input_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:94: operator(): block: [26,0,0], thread: [102,0,0] Assertion `input_val >= zero && input_va

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


#### MLP

In [16]:
import torch
from torch_geometric.utils import to_dense_adj

# Assume edge_index, num_nodes, and remove_edges_and_sample_optimized are defined
# Extract train graph adjacency matrix
num_n=train_edge_index.max().item() + 1
train_adj_matrix = to_dense_adj(train_edge_index, max_num_nodes=num_n)[0]  # Convert to dense adjacency matrix
train_adj_matrix = train_adj_matrix.to(torch.float32)  # Ensure float type for computations
train_adj_matrix = to_dense_adj(train_edge_index, max_num_nodes=num_nodes)[0] 
# Convert to SciPy sparse matrix
# Create the adjacency matrix from the edge list (train_edge_index)
#train_adj_matrix = to_dense_adj(train_edge_index, max_num_nodes=num_nodes)[0]

# Enforce symmetry (add the transpose to ensure both directions are captured)
train_adj_matrix = train_adj_matrix + train_adj_matrix.T

# Ensure that the diagonal entries are 1 (self-loops)
train_adj_matrix.fill_diagonal_(1.0)
train_adj_matrix = train_adj_matrix.clamp(max=1)
# Node features
if data.x is not None:
    X = data.x  # Use provided node features
else:
    X = torch.eye(num_nodes)  # Use identity matrix if featureless

# Normalize adjacency for training graph
A_tilde_train = normalize_adjacency_dense_gpu(train_adj_matrix)

# Initialize model
input_dim = X.shape[1]
hidden_dim = 32
latent_dim = 16
model = VGAE_MLP(input_dim, hidden_dim, latent_dim)

model = model.to('cuda')  # Move model to GPU if available
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [85]:

num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    Z, A_reconstructed, mu, log_sigma = model(X.to('cuda'), A_tilde_train.to('cuda'), edge_index=edge_index)
    
    # Clamp log_sigma to prevent extreme values
    log_sigma = torch.clamp(log_sigma, min=-18, max=18)
    
    # Compute loss
    loss = loss_function_mlp(train_adj_matrix.to('cuda'), A_reconstructed, mu, log_sigma, train_edge_index.to('cuda'))
    loss.backward()

    # Update parameters
    optimizer.step()

    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")



NameError: name 'edge_index' is not defined

#### Evaluation

In [21]:
import numpy as np
from sklearn.metrics import roc_auc_score

# Convert the reconstructed adjacency matrix to CPU if necessary
A_reconstructed = A_reconstructed.detach().cpu()

test_edges = torch.tensor(test_edges, dtype=torch.long)
test_non_edges = torch.tensor(test_non_edges, dtype=torch.long)
# Get the scores for test edges and test non-edges
test_edge_scores = A_reconstructed[test_edges[:, 0], test_edges[:, 1]].numpy()
test_non_edge_scores = A_reconstructed[test_non_edges[:, 0], test_non_edges[:, 1]].numpy()

# Combine scores and create labels
scores = np.concatenate([test_edge_scores, test_non_edge_scores])
labels = np.concatenate([np.ones(len(test_edge_scores)), np.zeros(len(test_non_edge_scores))])


/scratch-local/scur2863.9726859/ipykernel_844684/3338621847.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_edges = torch.tensor(test_edges, dtype=torch.long)
/scratch-local/scur2863.9726859/ipykernel_844684/3338621847.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_non_edges = torch.tensor(test_non_edges, dtype=torch.long)


In [17]:

import numpy as np
from sklearn.metrics import roc_auc_score

# Convert the reconstructed adjacency matrix to CPU if necessary
A_reconstructed = A_reconstructed.detach().cpu()

# Ensure test edges and non-edges are tensors
test_edges = torch.tensor(test_edges, dtype=torch.long)
test_non_edges = torch.tensor(test_non_edges, dtype=torch.long)

# Handle different decoder outputs
if A_reconstructed.dim() == 2:  
    # If A_reconstructed is a full adjacency matrix
    test_edge_scores = A_reconstructed[test_edges[:, 0], test_edges[:, 1]].numpy()
    test_non_edge_scores = A_reconstructed[test_non_edges[:, 0], test_non_edges[:, 1]].numpy()
else:
    # If A_reconstructed is a 1D tensor (edge probabilities only)
    test_edge_scores = A_reconstructed[:len(test_edges)].numpy()
    test_non_edge_scores = A_reconstructed[len(test_edges):].numpy()

# Combine scores and create labels
scores = np.concatenate([test_edge_scores, test_non_edge_scores])
labels = np.concatenate([np.ones(len(test_edge_scores)), np.zeros(len(test_non_edge_scores))])


In [18]:
from sklearn.metrics import roc_auc_score

# Assuming y_true contains actual edge labels (1 for edges, 0 for non-edges)
# and y_score contains the predicted scores for each pair of nodes
roc_auc = roc_auc_score(labels, scores)
print(f"ROC-AUC Score: {roc_auc}")


ROC-AUC Score: 0.6355253342373627


In [19]:
from sklearn.metrics import average_precision_score
ap_score = average_precision_score(labels, scores)
print(f"Average Precision (AP): {ap_score:.4f}")

Average Precision (AP): 0.6649


In [ ]:
0.5897 y 0.5980

### Patent dataset

In [11]:
from scipy.sparse import save_npz, load_npz
import numpy as np
import torch
from scipy.sparse import coo_matrix
from sklearn.model_selection import train_test_split

A = load_npz("combined_adj_small.npz")
X = load_npz("combined_features_matrix.npz")

X = torch.tensor(X.toarray(), dtype=torch.float32)

In [13]:
# Check if the matrix is symmetric
if (A != A.T).nnz == 0:  # If the number of non-zero elements in (A - A.T) is zero
    print("Matrix A is symmetric.")
else:
    print("Matrix A is not symmetric.")

Matrix A is symmetric.


In [14]:


def split_edges_and_sample(A, num_samples=None, test_size=0.1, val_size=0.05, random_state=42):
    """
    Efficiently splits edges and samples non-edges.
    
    Parameters:
    - A: scipy.sparse.coo_matrix (adjacency matrix)
    - num_samples: Number of non-edges to sample (adjust based on graph size)
    - test_size: Proportion of edges/non-edges for testing
    - val_size: Proportion of edges/non-edges for validation
    - random_state: Random seed for reproducibility
    
    Returns:
    - train_edge_index: Edge list for training
    - val_edges, test_edges: Validation and test edge lists
    - val_non_edges, test_non_edges: Validation and test non-edges
    - train_graph: Sparse matrix representing the training graph
    """
    A_coo = coo_matrix(A)
    edges = np.vstack((A_coo.row, A_coo.col)).T  # Extract edges
    num_nodes = A.shape[0]
    
    # Convert edges to a set for fast lookup
    existing_edges = set(map(tuple, edges))
    num_samples=len(edges)*0.15
    # Randomly sample non-edges
    np.random.seed(random_state)
    non_edges = set()
    while len(non_edges) < num_samples:
        i = np.random.randint(0, num_nodes)
        j = np.random.randint(0, num_nodes)
        if i != j and (i, j) not in existing_edges and (j, i) not in existing_edges:
            non_edges.add((i, j))
    #print(existing_edges)
    non_edges = np.array(list(non_edges))
    
    # Split edges into train, validation, and test sets
    train_edges, temp_edges = train_test_split(edges, test_size=(test_size + val_size), random_state=random_state)
    
    val_edges, test_edges = train_test_split(temp_edges, test_size=(test_size / (test_size + val_size)), random_state=random_state)
    
    # Split sampled non-edges into validation and test sets
    val_non_edges, test_non_edges = train_test_split(non_edges, test_size=(test_size / (test_size + val_size)), random_state=random_state)
    
    # Create a training graph (without val/test edges)
    train_graph = coo_matrix(
        (np.ones(len(train_edges)), (train_edges[:, 0], train_edges[:, 1])),
        shape=A.shape
    )
    train_graph = train_graph + train_graph.T  # Ensure symmetry
    
    # Convert to PyTorch tensors
    train_edge_index = torch.tensor(train_edges.T, dtype=torch.long)
    val_edges = torch.tensor(val_edges, dtype=torch.long)
    test_edges = torch.tensor(test_edges, dtype=torch.long)
    val_non_edges = torch.tensor(val_non_edges, dtype=torch.long)
    test_non_edges = torch.tensor(test_non_edges, dtype=torch.long)

    return train_edge_index, val_edges, test_edges, val_non_edges, test_non_edges, train_graph


In [15]:
#A = A + A.T  # Ensure symmetry for an undirected graph
#A[A > 1] = 1  # Remove duplicate edges

# Split edges and sample non-edges
train_edge_index, val_edges, test_edges, val_non_edges, test_non_edges, train_graph= split_edges_and_sample(A)

print("Train edge index shape:", train_edge_index.shape)
print("Number of validation edges:", len(val_edges))
print("Number of test edges:", len(test_edges))
print("Number of validation non-edges:", len(val_non_edges))
print("Number of test non-edges:", len(test_non_edges))


Train edge index shape: torch.Size([2, 44438])
Number of validation edges: 2614
Number of test edges: 5229
Number of validation non-edges: 2614
Number of test non-edges: 5229


In [16]:
def normalize_adjacency_sparse_gpu(A):
    """
    Normalize adjacency matrix on GPU using sparse matrices.
    A: Sparse adjacency matrix (torch.sparse.FloatTensor).
    """
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    A = A.to(device)  # Move to GPU if available
    
    # Ensure self-loops (can be done with sparse matrices too)
    #eye = torch.eye(A.size(0), device=A.device).to_sparse()
    #A = A + eye
    
    # Degree vector (sparse sum)
    row_sum = torch.sum(A, dim=1) # Sparse sum and convert to dense
    
    # Avoid division by zero by adding a small epsilon
    D_inv_sqrt = torch.diag(1.0 / torch.sqrt(row_sum + 1e-10))

    # Normalize adjacency
    normalized_A = D_inv_sqrt @ A @ D_inv_sqrt
    
    # Enforce symmetry
    normalized_A = (normalized_A + normalized_A.T) / 2.0
    
    return normalized_A


#### MLP

In [17]:
import torch
import torch_sparse
from torch_geometric.utils import to_scipy_sparse_matrix
from torch_geometric.utils import to_dense_adj

# Assume edge_index, num_nodes, and remove_edges_and_sample_optimized are defined
# Extract train graph adjacency matrix

# Extract train graph adjacency matrix
num_nodes =  max(train_edge_index[0].max(), train_edge_index[1].max()) + 1


train_adj_matrix = to_dense_adj(train_edge_index, max_num_nodes=num_nodes)[0] 
# Convert to SciPy sparse matrix


#train_adj_matrix = to_scipy_sparse_matrix(train_edge_index, num_nodes=num_nodes)
# Ensure adjacency matrix is sparse on GPU
#train_adj_matrix = train_adj_matrix.to_sparse().to(device)  # Move sparse matrix to GPU

# Convert directly to a PyTorch sparse tensor
#train_adj_matrix = torch.tensor(train_adj_matrix.toarray(), dtype=torch.float32) 
# Ensure adjacency matrix is sparse on GPU
#train_adj_matrix = train_adj_matrix.to_sparse().to('cuda')  # Move sparse matrix to GPU
#train_adj_matrix = torch.sparse_coo_tensor(indices, values, size=(num_nodes, num_nodes), dtype=torch.float32).to(device)

# Normalize adjacency for training graph
A_tilde_train = normalize_adjacency_dense_gpu(train_adj_matrix.to(torch.float32))  # Normalize

#train_adj_matrix = train_adj_matrix.to('cuda')
#A_tilde_train = normalize_adjacency_dense_gpu(train_adj_matrix)


# Initialize model
input_dim = X.shape[1]
hidden_dim = 64
latent_dim = 32
model = VGAE_MLP(input_dim, hidden_dim, latent_dim)

model = model.to('cuda')  # Move model to GPU if available
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

/home/scur2863/graphs/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor(3., device='cuda:0')
tensor([0.5774, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000], device='cuda:0')


In [ ]:

num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    Z, A_reconstructed, mu, log_sigma = model(X.to('cuda'), A_tilde_train.to('cuda'), edge_index=train_edge_index)
    
    # Clamp log_sigma to prevent extreme values
    log_sigma = torch.clamp(log_sigma, min=-18, max=18)
    
    # Compute loss
    loss = loss_function_mlp(train_adj_matrix.to('cuda'), A_reconstructed, mu, log_sigma, train_edge_index.to('cuda'))
    loss.backward()

    # Update parameters
    optimizer.step()

    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")



In [58]:
A_tilde_train = normalize_adjacency_dense_gpu(train_adj_matrix)
A_tilde_train = A_tilde_train.to(torch.float32)

Matrix A is on device: cuda:0
A after adding self-loops: torch.Size([17859, 17859])
Row sums: torch.Size([17859])
D_inv_sqrt: torch.Size([17859, 17859])
Normalized A: torch.Size([17859, 17859])
Final normalized A: torch.Size([17859, 17859])


In [60]:
D = torch.diag(A_tilde_train.sum(dim=1).clamp(min=1).pow(-0.5))
A_tilde_train = D @ A_tilde_train @ D


In [61]:
A_tilde_train = A_tilde_train / A_tilde_train.sum(dim=1, keepdim=True).clamp(min=1)


In [39]:
is_symmetric = torch.allclose(train_adj_matrix, train_adj_matrix.T, atol=1e-6)
print("Is A_tilde_train symmetric?", is_symmetric)

Is A_tilde_train symmetric? True


In [26]:
is_symmetric = torch.allclose(A_tilde_train, A_tilde_train.T, atol=1e-6)
print("Is A_tilde_train symmetric?", is_symmetric)

Is A_tilde_train symmetric? True


In [20]:

for i in train_adj_matrix[0]:
    if i!=0:
        print(i)

tensor(1.)
tensor(1.)


In [21]:
for i in A_tilde_train[0]:
    if i!=0:
        print(i)

tensor(0.5000, device='cuda:0')
tensor(0.0928, device='cuda:0')


In [93]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Layer: {name} | Weights: {param.data}")


Layer: encoder.gcn1.weight | Weights: tensor([[-0.5165, -0.7178,  0.1181,  ..., -1.4715, -0.0287,  0.2721],
        [ 0.4688,  1.2197, -2.1986,  ..., -0.7330,  2.5639, -2.0083],
        [ 0.6753,  0.8676,  1.9209,  ..., -0.5977, -0.2254,  0.2115],
        ...,
        [ 0.6558, -0.4602, -0.8751,  ..., -0.9342, -1.1248, -1.3540],
        [ 1.0623, -0.6393, -0.4553,  ...,  0.3842,  0.6187, -0.0309],
        [-0.6848, -0.2271,  0.1306,  ...,  0.3308,  0.1022, -1.2303]],
       device='cuda:0')
Layer: encoder.gcn2_mu.weight | Weights: tensor([[-0.4331, -0.6110, -0.2166, -1.6117, -1.2528, -0.2021, -0.0374,  0.4680,
          0.2090,  0.2320, -2.9715, -1.7069, -1.4590,  0.1886, -1.2897,  0.7148],
        [-1.3360,  0.7544, -0.2892,  0.1659, -0.0346,  1.4108, -1.3753, -0.3699,
          2.5905,  1.6400, -0.6388, -1.6709, -1.2031, -2.4951,  1.5671, -0.1564],
        [ 0.6477,  2.0286, -0.7522,  0.2703,  2.2812, -1.1301,  0.1586, -1.0083,
          2.0195,  0.6130, -0.5320, -0.3704, -0.5705, -0

In [ ]:
Layer: decoder.weight | Weights: tensor([ 0.7694,  0.7375,  0.7400,  0.8426,  0.6726,  0.5184,  0.8114, -0.0369,
         0.8284,  0.7812,  0.6179,  0.5305,  0.7229,  0.0629,  0.1734,  0.5006]

In [ ]:
Layer: decoder.weight | Weights: tensor([0.2511, 0.4997, 0.5713, 0.3977, 0.4057, 0.6872, 0.3339, 0.1781, 0.3881,
        0.8694, 0.3839, 0.3271, 0.4893, 0.7137, 0.7647, 0.5228],

In [19]:
row_sums=A_tilde_train.sum(dim=1)
print("Row sum min:", row_sums.min().item())
print("Row sum max:", row_sums.max().item())
print("Row sum mean:", row_sums.mean().item())


Row sum min: 0.2660941779613495
Row sum max: 21.876142501831055
Row sum mean: 0.6087040901184082


In [17]:
def split_edges_and_sample(A, num_samples=None, test_size=0.1, val_size=0.05, random_state=42):
    """
    Efficiently splits edges and samples non-edges.
    
    Parameters:
    - A: scipy.sparse.coo_matrix (adjacency matrix)
    - num_samples: Number of non-edges to sample (adjust based on graph size)
    - test_size: Proportion of edges/non-edges for testing
    - val_size: Proportion of edges/non-edges for validation
    - random_state: Random seed for reproducibility
    
    Returns:
    - train_edge_index: Edge list for training
    - val_edges, test_edges: Validation and test edge lists
    - val_non_edges, test_non_edges: Validation and test non-edges
    - train_graph: Sparse matrix representing the training graph
    """
    A_coo = coo_matrix(A)
    edges = np.vstack((A_coo.row, A_coo.col)).T  # Extract edges
    num_nodes = A.shape[0]
    
    # Normalize edge representation to avoid both (i, j) and (j, i)
    edges = np.array([tuple(sorted((i, j))) for i, j in edges])
    existing_edges = set(map(tuple, edges))
    
    # Sample non-edges
    if num_samples is None:
        num_samples = int(len(edges) * 0.15)  # Sample 15% of edges as non-edges by default
    
    np.random.seed(random_state)
    non_edges = set()
    while len(non_edges) < num_samples:
        i = np.random.randint(0, num_nodes)
        j = np.random.randint(0, num_nodes)
        if i != j:
            edge = tuple(sorted((i, j)))
            if edge not in existing_edges:
                non_edges.add(edge)
    
    non_edges = np.array(list(non_edges))
    
    # Split edges into train, validation, and test sets
    train_edges, temp_edges = train_test_split(edges, test_size=(test_size + val_size), random_state=random_state)
    val_edges, test_edges = train_test_split(temp_edges, test_size=(test_size / (test_size + val_size)), random_state=random_state)
    
    # Split sampled non-edges into validation and test sets
    val_non_edges, test_non_edges = train_test_split(non_edges, test_size=(test_size / (test_size + val_size)), random_state=random_state)
    
    # Create a training graph (without val/test edges)
    train_graph = coo_matrix(
        (np.ones(len(train_edges)), (train_edges[:, 0], train_edges[:, 1])),
        shape=A.shape
    )
    train_graph = train_graph + train_graph.T  # Ensure symmetry
    
    # Convert to PyTorch tensors
    train_edge_index = torch.tensor(train_edges.T, dtype=torch.long)
    val_edges = torch.tensor(val_edges, dtype=torch.long)
    test_edges = torch.tensor(test_edges, dtype=torch.long)
    val_non_edges = torch.tensor(val_non_edges, dtype=torch.long)
    test_non_edges = torch.tensor(test_non_edges, dtype=torch.long)

    return train_edge_index, val_edges, test_edges, val_non_edges, test_non_edges, train_graph


In [94]:

import torch
import torch_sparse
from torch_geometric.utils import to_scipy_sparse_matrix
from torch_geometric.utils import to_dense_adj

# Assume edge_index, num_nodes, and remove_edges_and_sample_optimized are defined
# Extract train graph adjacency matrix

# Extract train graph adjacency matrix
num_nodes = train_edge_index.max().item() + 1

train_adj_matrix = to_dense_adj(train_edge_index, max_num_nodes=num_nodes)[0] 
# Convert to SciPy sparse matrix
# Create the adjacency matrix from the edge list (train_edge_index)
#train_adj_matrix = to_dense_adj(train_edge_index, max_num_nodes=num_nodes)[0]

# Enforce symmetry (add the transpose to ensure both directions are captured)
train_adj_matrix = train_adj_matrix + train_adj_matrix.T

# Ensure that the diagonal entries are 1 (self-loops)
train_adj_matrix.fill_diagonal_(1.0)
train_adj_matrix = train_adj_matrix.clamp(max=1)
#train_adj_matrix = to_scipy_sparse_matrix(train_edge_index, num_nodes=num_nodes)

# Convert directly to a PyTorch sparse tensor
#train_adj_matrix = torch.tensor(train_adj_matrix.toarray(), dtype=torch.float32) 

# Normalize adjacency for training graph
A_tilde_train = normalize_adjacency_dense_gpu(train_adj_matrix.to(torch.float32))  # Normalize
#D = torch.diag(train_adj_matrix.sum(dim=1).clamp(min=1).pow(-0.5))
#A_tilde_train = D @ train_adj_matrix @ D


# Initialize model
input_dim = X.shape[1]
hidden_dim = 32
latent_dim = 16
model = VGAE_W(input_dim, hidden_dim, latent_dim)

model = model.to('cuda')  # Move model to GPU if available
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

tensor(2., device='cuda:0')
tensor([0.7071, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000], device='cuda:0')


In [89]:
# Training OLD
num_epochs = 400
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    Z, A_reconstructed, mu, log_sigma = model(X.to('cuda'), A_tilde_train.to('cuda'))
    
    # Clamp log_sigma to prevent extreme values
    log_sigma = torch.clamp(log_sigma, min=-18, max=18)
    
    # Compute loss
    loss = loss_function(train_adj_matrix.to('cuda'), A_reconstructed.to('cuda'), mu, log_sigma)
    loss.backward()

    # Update parameters
    optimizer.step()

    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

Epoch 1, Loss: 106998472.0
Epoch 2, Loss: 104572016.0
Epoch 3, Loss: 102433448.0
Epoch 4, Loss: 100580400.0
Epoch 5, Loss: 98991800.0
Epoch 6, Loss: 97562920.0
Epoch 7, Loss: 96296664.0
Epoch 8, Loss: 95181576.0
Epoch 9, Loss: 94016464.0
Epoch 10, Loss: 93077608.0
Epoch 11, Loss: 92143856.0
Epoch 12, Loss: 91420328.0
Epoch 13, Loss: 90531512.0
Epoch 14, Loss: 89728112.0
Epoch 15, Loss: 89040912.0
Epoch 16, Loss: 88336384.0
Epoch 17, Loss: 87677632.0
Epoch 18, Loss: 87146328.0
Epoch 19, Loss: 86603424.0
Epoch 20, Loss: 86117480.0
Epoch 21, Loss: 85684856.0
Epoch 22, Loss: 85355400.0
Epoch 23, Loss: 85010264.0
Epoch 24, Loss: 84691120.0
Epoch 25, Loss: 84410040.0
Epoch 26, Loss: 84201440.0
Epoch 27, Loss: 84008528.0
Epoch 28, Loss: 83836040.0
Epoch 29, Loss: 83664336.0
Epoch 30, Loss: 83478576.0
Epoch 31, Loss: 83358520.0
Epoch 32, Loss: 83277936.0
Epoch 33, Loss: 83197328.0
Epoch 34, Loss: 83136488.0
Epoch 35, Loss: 83016320.0
Epoch 36, Loss: 82929000.0
Epoch 37, Loss: 82849368.0
Epoch 

In [90]:

import numpy as np
from sklearn.metrics import roc_auc_score

# Convert the reconstructed adjacency matrix to CPU if necessary
A_reconstructed = A_reconstructed.detach().cpu()

# Ensure test edges and non-edges are tensors
test_edges = torch.tensor(test_edges, dtype=torch.long)
test_non_edges = torch.tensor(test_non_edges, dtype=torch.long)

# Handle different decoder outputs
if A_reconstructed.dim() == 2:  
    # If A_reconstructed is a full adjacency matrix
    test_edge_scores = A_reconstructed[test_edges[:, 0], test_edges[:, 1]].numpy()
    test_non_edge_scores = A_reconstructed[test_non_edges[:, 0], test_non_edges[:, 1]].numpy()
else:
    # If A_reconstructed is a 1D tensor (edge probabilities only)
    test_edge_scores = A_reconstructed[:len(test_edges)].numpy()
    test_non_edge_scores = A_reconstructed[len(test_edges):].numpy()

# Combine scores and create labels
scores = np.concatenate([test_edge_scores, test_non_edge_scores])
labels = np.concatenate([np.ones(len(test_edge_scores)), np.zeros(len(test_non_edge_scores))])


/scratch-local/scur2863.9735026/ipykernel_999491/3313984776.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_edges = torch.tensor(test_edges, dtype=torch.long)
/scratch-local/scur2863.9735026/ipykernel_999491/3313984776.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_non_edges = torch.tensor(test_non_edges, dtype=torch.long)


In [91]:
from sklearn.metrics import roc_auc_score

# Assuming y_true contains actual edge labels (1 for edges, 0 for non-edges)
# and y_score contains the predicted scores for each pair of nodes
roc_auc = roc_auc_score(labels, scores)
print(f"ROC-AUC Score: {roc_auc}")


ROC-AUC Score: 0.6941392138324446


In [92]:
from sklearn.metrics import average_precision_score
ap_score = average_precision_score(labels, scores)
print(f"Average Precision (AP): {ap_score:.4f}")

Average Precision (AP): 0.7416


In [ ]:
mpl 0.4957 y 0.11

In [ ]:
inner product 0.5692 y 0.5341

## GAE

In [52]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GCNLayer(nn.Module):
    """
    Graph Convolutional Layer for GAE
    """
    def __init__(self, in_channels, out_channels):
        super(GCNLayer, self).__init__()
        self.weight = nn.Parameter(torch.randn(in_channels, out_channels))
        self.bias = nn.Parameter(torch.zeros(out_channels))

    def forward(self, X, A):
        """
        X: Node features matrix (num_nodes, input_dim)
        A: Adjacency matrix (num_nodes, num_nodes)
        """
        support = torch.matmul(X, self.weight)  # Apply linear transformation
        output = torch.matmul(A, support)  # Aggregate neighbor information
        return output + self.bias  # Add bias term

class GAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(GAE, self).__init__()
        # Encoder layers
        self.gcn1 = GCNLayer(input_dim, hidden_dim)
        self.gcn2 = GCNLayer(hidden_dim, latent_dim)

    def forward(self, X, A_tilde):
        """
        X: Node feature matrix (num_nodes, input_dim)
        A_tilde: Normalized adjacency matrix (num_nodes, num_nodes)
        """
        # Graph convolution layer 1
        H = F.relu(self.gcn1(X, A_tilde))  # First GCN layer with ReLU activation
        
        # Graph convolution layer 2
        Z = self.gcn2(H, A_tilde)  # Second GCN layer for embeddings Z
        
        # Normalize the embeddings (optional, based on your specific use case)
        Z = F.normalize(Z, p=2, dim=1)  # Normalize each row of Z to have unit length
        
        # Reconstruct the adjacency matrix
        A_reconstructed = torch.sigmoid(torch.matmul(Z, Z.T))  # Reconstruct the adjacency matrix
        
        return Z, A_reconstructed


In [53]:
def loss_function(A, A_reconstructed):
    """
    A: True adjacency matrix (num_nodes, num_nodes)
    A_reconstructed: Reconstructed adjacency matrix (num_nodes, num_nodes)
    """
    # Flatten both matrices to make them compatible for loss computation
    A = A.view(-1)
    A_reconstructed = A_reconstructed.view(-1)
    
    # Compute binary cross-entropy loss
    return F.binary_cross_entropy(A_reconstructed, A)


In [55]:
import torch
from torch_geometric.utils import to_dense_adj

# Assume edge_index, num_nodes, and remove_edges_and_sample_optimized are defined
# Extract train graph adjacency matrix
train_adj_matrix = to_dense_adj(train_edge_index, max_num_nodes=num_nodes)[0]  # Convert to dense adjacency matrix
train_adj_matrix = train_adj_matrix.to(torch.float32)  # Ensure float type for computations

# Node features
#if data.x is not None:
 #   X = data.x  # Use provided node features
#else:
X = torch.eye(num_nodes)  # Use identity matrix if featureless

# Normalize adjacency for training graph
A_tilde = normalize_adjacency_dense_gpu(train_adj_matrix)

In [59]:
# Hyperparameters
input_dim = X.shape[1]  # Number of features per node
hidden_dim = 32  # Hidden layer size
latent_dim = 16  # Latent space size (embedding dimension)
num_epochs = 200
learning_rate = 0.01
device='cuda'
# Initialize model and optimizer
model = GAE(input_dim, hidden_dim, latent_dim).to(device) 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training Loop
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    Z, A_reconstructed = model(X.to(device), train_adj_matrix.to(device))
    
    # Compute loss
    loss = loss_function(train_adj_matrix.to(device), A_reconstructed)
    
    # Backward pass
    loss.backward()
    
    # Update parameters
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")


Epoch 10/200, Loss: 0.7251855731010437
Epoch 20/200, Loss: 0.7126666307449341
Epoch 30/200, Loss: 0.709176778793335
Epoch 40/200, Loss: 0.7057015299797058
Epoch 50/200, Loss: 0.7053874731063843
Epoch 60/200, Loss: 0.7044334411621094
Epoch 70/200, Loss: 0.7040550112724304
Epoch 80/200, Loss: 0.7036513090133667
Epoch 90/200, Loss: 0.7033585906028748
Epoch 100/200, Loss: 0.7030957937240601
Epoch 110/200, Loss: 0.7028746604919434
Epoch 120/200, Loss: 0.702678382396698
Epoch 130/200, Loss: 0.7025027871131897
Epoch 140/200, Loss: 0.7023438811302185
Epoch 150/200, Loss: 0.7021982073783875
Epoch 160/200, Loss: 0.702064037322998
Epoch 170/200, Loss: 0.7019408345222473
Epoch 180/200, Loss: 0.7018279433250427
Epoch 190/200, Loss: 0.7017245888710022
Epoch 200/200, Loss: 0.7016303539276123


In [60]:
import numpy as np
from sklearn.metrics import roc_auc_score, average_precision_score


# Convert the reconstructed adjacency matrix to CPU if necessary
A_reconstructed = A_reconstructed.detach().cpu()

test_edges = torch.tensor(test_edges, dtype=torch.long)
test_non_edges = torch.tensor(test_non_edges, dtype=torch.long)
# Get the scores for test edges and test non-edges
test_edge_scores = A_reconstructed[test_edges[:, 0], test_edges[:, 1]].numpy()
test_non_edge_scores = A_reconstructed[test_non_edges[:, 0], test_non_edges[:, 1]].numpy()

# Combine scores and create labels
scores = np.concatenate([test_edge_scores, test_non_edge_scores])
labels = np.concatenate([np.ones(len(test_edge_scores)), np.zeros(len(test_non_edge_scores))])

# Assuming y_true contains actual edge labels (1 for edges, 0 for non-edges)
# and y_score contains the predicted scores for each pair of nodes
roc_auc = roc_auc_score(labels, scores)
print(f"ROC-AUC Score: {roc_auc}")

ap_score = average_precision_score(labels, scores)
print(f"Average Precision (AP): {ap_score:.4f}")

ROC-AUC Score: 0.8473404554648748
Average Precision (AP): 0.8702


/scratch-local/scur2863.9595676/ipykernel_1494540/598414332.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_edges = torch.tensor(test_edges, dtype=torch.long)
/scratch-local/scur2863.9595676/ipykernel_1494540/598414332.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_non_edges = torch.tensor(test_non_edges, dtype=torch.long)


#### Appendix


In [ ]:
from gensim.models import Word2Vec

def deepwalk(graph, num_walks=10, walk_length=80, window_size=10):
    walks = []
    for node in graph.nodes():
        for _ in range(num_walks):
            walk = [node]
            for _ in range(walk_length):
                neighbors = list(graph.neighbors(walk[-1]))
                walk.append(np.random.choice(neighbors))
            walks.append(walk)
    model = Word2Vec(walks, vector_size=128, window=window_size, min_count=1, sg=1)
    return model.wv

In [ ]:
from sklearn.cluster import SpectralClustering

sc = SpectralClustering(n_clusters=2, affinity='precomputed')
sc_embeddings = sc.fit_predict(adjacency_matrix)


In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score

# Convert the reconstructed adjacency matrix to CPU if necessary
A_reconstructed = A_reconstructed.detach().cpu()

test_edges = torch.tensor(test_edges, dtype=torch.long)
test_non_edges = torch.tensor(test_non_edges, dtype=torch.long)
# Get the scores for test edges and test non-edges
test_edge_scores = A_reconstructed[test_edges[:, 0], test_edges[:, 1]].numpy()
test_non_edge_scores = A_reconstructed[test_non_edges[:, 0], test_non_edges[:, 1]].numpy()

# Combine scores and create labels
scores = np.concatenate([test_edge_scores, test_non_edge_scores])
labels = np.concatenate([np.ones(len(test_edge_scores)), np.zeros(len(test_non_edge_scores))])
